In [1]:
useMock=True

In [2]:
from cffi import FFI
ffi = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
ffi.cdef(myHeaders)
with open('extracted_headers/safe_c_ffi_funcs.h','r') as f:
    funs=f.read()
ffi.cdef(funs)
lib=ffi.dlopen('../compiled_binaries/libsafe_app' + useMock*'_mock' + '.so')

---

maybe it makes most sense to really define the data types by hand and just check them for changes with updates on the api - not entirely sure

- rid

---

```c
typedef unsigned long int uintptr_t;

typedef struct {
 char* id;
 char* scope;
 char* name;
 char* vendor;
} AppExchangeInfo;
```

In [3]:
class AppExchangeInfo:
    def __init__(self,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
        self.id = ffi.new('char[]',id)
        self.scope = ffi.new('char[]',scope)
        self.name = ffi.new('char[]',name)
        self.vendor = ffi.new('char[]',vendor)

        self.entity = ffi.new('AppExchangeInfo *',[self.id,self.scope,self.name,self.vendor])

In [4]:
def AppExchangeInfo(id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
    id = ffi.new('char[]',id)
    scope = ffi.new('char[]',scope)
    name = ffi.new('char[]',name)
    vendor = ffi.new('char[]',vendor)

    myStruct = ffi.new('AppExchangeInfo *',[id,scope,name,vendor])
    
    return myStruct, [id, scope, name, vendor]

newExChangeInfo,infopayload = AppExchangeInfo(b'0.0.2',name=b'pyTest',vendor=b'rid')

In [5]:
newExChangeInfo,infopayload = AppExchangeInfo(id=b'net.maidsafe.tutorials.nodejs',scope=b'0.0.1',
                                              name=b'PyTest never ever',vendor=b'riddim from over there')

In [6]:
ffi.string(newExChangeInfo.vendor)

b'riddim from over there'

In [7]:
ffi.string(newExChangeInfo.name)

b'PyTest never ever'

```c
typedef struct {
 _Bool read;
 _Bool insert;
 _Bool update;
 _Bool delete;
 _Bool manage_permissions;
} PermissionSet;
```

In [8]:
def PermissionSet(read=True,insert=True,update=True,delete=True,manage_permissions=True):
    return ffi.new('PermissionSet *',[read,insert,update,delete,manage_permissions])

In [9]:
newPermissionSet = PermissionSet()

```c
typedef struct {
 char* cont_name;
 PermissionSet access;
} ContainerPermissions;
```

In [10]:
def ContainerPermissions(name=b'noName',access=None):
    containerName = ffi.new('char[]',name)
    if not access:
        access = PermissionSet()
    container = ffi.new('ContainerPermissions *',[containerName,access[0]])
    
    return container, [containerName,access]

In [11]:
newPermissions,permission_addon=ContainerPermissions(b'containerPy')

In [12]:
newPermissions.access.insert

True

```c
typedef struct {
 AppExchangeInfo app;
 _Bool app_container;
 ContainerPermissions* containers;
 uintptr_t containers_len;
 uintptr_t containers_cap;
} AuthReq;
```

In [13]:
def AuthReq(permissions,containers_len,containers_cap,id=b'noId',scope=b'pythonscript',
            name=b'noName',vendor=b'nobody',app_container=True):
    
    newExChangeInfo,infopayload = AppExchangeInfo(id,scope,name,vendor)
    
    authReq = ffi.new('AuthReq *',[newExChangeInfo[0],app_container,permissions,containers_len,containers_cap])
    
    return authReq, [newExChangeInfo,infopayload]

In [14]:
myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'rid never work',app_container=True)

In [15]:
myAuth,addData=AuthReq(ffi.NULL,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'rid never work',app_container=False)

In [16]:
@ffi.callback("void(void *, FfiResult *)")
def feedback(userData, result):
    print(userData)
    print(result.error_code)

In [17]:
lib.app_set_additional_search_path(ffi.new('char[]',b'../compiled_binaries/'),ffi.NULL,feedback)

<cdata 'void *' NULL>
0


@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, noClueWhatNumber, resultString):
    print(userData)
    print(result)
    print(noClueWhatNumber)
    print(resultString)

lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

myAuth.containers.access.insert=True

myAuth.containers.access.delete=True

myAuth.containers.access.read=True

In [18]:
ffi.string(myAuth.app.name)

b'PyTest'

In [19]:
ffi.string(myAuth.app.id)

b'ridsTest'

In [20]:
encodedAuth=None

In [21]:
@ffi.callback("void(void *, FfiResult *, unsigned long int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [22]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [23]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
0
614708766
b'AAAAAB62oyQAAAAACAAAAAAAAAByaWRzVGVzdAEMAAAAAAAAAHB5dGhvbnNjcmlwdAYAAAAAAAAAUHlUZXN0DgAAAAAAAAByaWQgbmV2ZXIgd29yawAAAAAAAAAAAA'


In [24]:
b'safe-auth://' + encodedAuth

b'safe-auth://AAAAAB62oyQAAAAACAAAAAAAAAByaWRzVGVzdAEMAAAAAAAAAHB5dGhvbnNjcmlwdAYAAAAAAAAAUHlUZXN0DgAAAAAAAAByaWQgbmV2ZXIgd29yawAAAAAAAAAAAA'

In [25]:
import base64

In [26]:
schemeName=b'safe-'+base64.encodebytes(ffi.string(myAuth.app.id))[:-2]

In [27]:

sysUri = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
sysUri.cdef(myHeaders)
with open('extracted_headers/system_uri.h','r') as f:
    funs=f.read()
sysUri.cdef(funs)
lib2=sysUri.dlopen('../compiled_binaries/libsystem_uri.so')

In [28]:
@sysUri.callback("void(void *, FfiResult *)")
def installerFeedback(userData, result):
    print(userData)
    print(result.error_code)

```c
void install(char* bundle, 
             char* vendor, 
             char* name, 
             char* exec_args, 
             uint_least64_t exec_args_len, 
             char* icon, 
             char* schemes, 
             void* user_data, 
             void (*o_cb)(void* user_data, FfiResult* result));
```

In [29]:
print(ffi.string(myAuth.app.name))
print(ffi.string(myAuth.app.vendor))
print(ffi.string(myAuth.app.scope))

b'PyTest'
b'rid never work'
b'pythonscript'


In [30]:
import os
print(os.getcwd())

/home/yomuga/maidsafe/pySafe/pySafe


In [31]:
pathToHandler=(os.getcwd()+'/uriHandler.py').encode()

In [32]:
pathToHandler

b'/home/yomuga/maidsafe/pySafe/pySafe/uriHandler.py'

In [33]:
bundle=sysUri.new('char[]',ffi.string(myAuth.app.id))
vendor=sysUri.new('char[]',ffi.string(myAuth.app.vendor))
name=sysUri.new('char[]',ffi.string(myAuth.app.name))
exec_args=sysUri.new('char[]',b'python')
exec_args2=sysUri.new('char[]',pathToHandler)
exec_args_len=2
icon=sysUri.new('char[]',b'')
schemes=sysUri.new('char[]',schemeName)
user_data=sysUri.NULL

In [34]:
exec_args_1 = sysUri.new('char*[]',[exec_args,exec_args2])

In [35]:
lib2.install(bundle,vendor,name,exec_args_1 ,exec_args_len,icon,schemes,user_data,installerFeedback)

<cdata 'void *' NULL>
0


```c 
void open_uri(char* uri, 
              void* user_data, 
              void (*o_cb)(void* user_data, FfiResult* result));
```

In [36]:
@sysUri.callback("void(void *, FfiResult *)")
def openerFeedback(userData, result):
    global res
    res = result
    print(userData)
    print(result.error_code)

In [37]:
ffi.string(myAuth.app.name)

b'PyTest'

In [38]:
ffi.string(myAuth.app.vendor)

b'rid never work'

In [39]:
ffi.string(myAuth.app.name)

b'PyTest'

In [40]:
uri=sysUri.new('char[]',b'safe-auth://' + encodedAuth)
user_data=sysUri.NULL

In [41]:
lib2.open_uri(uri,user_data,openerFeedback)

<cdata 'void *' NULL>
0


In [42]:
def printCrustConfig(configAddress,configlength=42):
    lali=[]
    for i in range(configlength):
        lali.append(configAddress[i])
    print(lali)

```c 
void decode_ipc_msg(char* msg, 
                    void* user_data, 
                    void (*o_auth)(void* user_data, uint32_t req_id, AuthGranted* auth_granted), 
                    void (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* serialised_cfg, 
                                            uintptr_t serialised_cfg_len), 
                    void (*o_containers)(void* user_data, uint32_t req_id), 
                    void (*o_share_mdata)(void* user_data, uint32_t req_id), 
                    void (*o_revoked)(void* user_data), 
                    void (*o_err)(void* user_data, FfiResult* result, uint32_t req_id));
```

In [43]:
import time

In [44]:
grantedAuth=None
innerVals=None
anotherVal=None
bootstrap_config=None
@ffi.callback("void(void *, uint32_t, AuthGranted *)")
def o_auth_decode_ipc_msg(userData, req_id, auth_granted):
    global grantedAuth
    global innerVals
    global anotherVal
    global bootstrap_config
    printCrustConfig(auth_granted.bootstrap_config)
    grantedAuth = ffi.new('AuthGranted *', auth_granted[0])
    
    tmpConfig=b''
    for i in range(auth_granted.bootstrap_config_len):
        tmpConfig+=chr(auth_granted.bootstrap_config[i]).encode()
    print(tmpConfig)
    bootstrap_config = ffi.new('unsigned char[]',tmpConfig)
    print('o_auth')
    print(userData)
    print(req_id)
    print(bootstrap_config)

In [45]:
@ffi.callback("void(void *, uint32_t, uint8_t *, uintptr_t)")
def o_unregistered_decode_ipc_msg(userData, req_id, serialised_cfg, serialised_cfg_len):
    print('o_unregistered')
    print(req_id)

In [46]:
@ffi.callback("void(void *, uint32_t)")
def o_containers_decode_ipc_msg(userData, req_id):
    print('o_containers')
    print(req_id)

In [47]:
@ffi.callback("void(void *, uint32_t)")
def o_share_mdata_decode_ipc_msg(userData, req_id):
    print('o_share')
    print(req_id)

In [48]:
@ffi.callback("void(void *)")
def o_revoked_decode_ipc_msg(userData):
    print('o_revoked')
    print(req_id)

In [49]:
@ffi.callback("void(void *, FfiResult *, uint32_t)")
def o_error_decode_ipc_msg(userData, result, req_id):
    global res
    res = result
    print('o_err')
    print(userData)
    print(req_id)
    print(result.error_code)
    print(ffi.string(result.description))

In [141]:
with open('answer','r') as f:
    answer=f.readlines()

In [142]:
import re
data=re.compile("'safe-.+?:(.+?)'")

In [143]:
my=data.findall(answer[-1])

In [144]:
msg = ffi.new('char[]',my[-1].encode())

In [145]:
my[-1]

'AQAAAB62oyQAAAAAAAAAACAAAAAAAAAAEeCYHJxe9sT7ZyVxYD4-xSS8Av7G56TiHTsqnw2wYj4gAAAAAAAAAFL0j9bKvQVvEzQa-BsPw-QkCx1qQEqYMua6jszaSl3eIAAAAAAAAAD6Dxg2B-CSZdIibROWC3RbjC0wotJzsRNCeEYlTWFOokAAAAAAAAAA0kJZMqIR7R0Rfg2Q_Cs782_dupa52UFEQUhbfLR12DT6Dxg2B-CSZdIibROWC3RbjC0wotJzsRNCeEYlTWFOoiAAAAAAAAAAAADpv17qDNqIf50i7C_3WgavJ58vfUH1NkpvdJC7u34gAAAAAAAAAGfJ9GqFr5SPQd72NBgM5EuASWoFxyUcCkszyeUbaLV_AAAAAAAAAAAAAAAAAAAAADBCRfxZCM_xIxo66CR-hIsDLQ9V7JFNHeDxe7ivWxc4mDoAAAAAAAAYAAAAAAAAABgvDc9xUwkhVg4QsGWREEKRYFU0KURH1QAAAAAAAAAA'

In [146]:
lib.decode_ipc_msg(msg,ffi.NULL,o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 80, 0, 0, 0, 0, 0, 0, 0, 33, 4, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 32, 20]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
o_auth
<cdata 'void *' NULL>
614708766
<cdata 'unsigned char[]' owning 17 bytes>


----

now the app

In [147]:
myApp = None

In [148]:
@ffi.callback("void(void*)")
def disconnect_app_registered(user_data):
    print('disconnected - callback from app_registered')

In [149]:
myApp

In [150]:
innerData=None
@ffi.callback("void(void* , FfiResult* , App* app)")
def result_app_registered(user_data, result, app):
    print('results')
    global myApp
    if app != ffi.NULL:
        myApp = ffi.new('App *',app[0])
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

In [151]:
grantedAuth.bootstrap_config

<cdata 'unsigned char *' 0x560215408120>

In [152]:
bootstrap_config

<cdata 'unsigned char[]' owning 17 bytes>

In [153]:
lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

results
error code: 
-1
b'Serialisation error'


In [154]:
grantedAuth.bootstrap_config = bootstrap_config

In [155]:
grantedAuth.bootstrap_config

<cdata 'unsigned char *' 0x560215114690>

In [156]:
lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

results
error code: 
0


In [65]:
myApp[0]

<cdata 'void *' 0x55acbc6d6c20>

In [67]:
grantedAuth.bootstrap_config_cap

16

In [94]:
lib.app_registered(myAuth.app.name,testDataAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

results
140247862083583
b'Serialisation error'


In [148]:
def printCrustConfig(configAddress,configlength=42):
    lali=[]
    for i in range(configlength):
        lali.append(configAddress[i])
    print(lali)

In [149]:
printCrustConfig(testDataAuth.bootstrap_config)

[0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 1, 0, 1, 0]


In [136]:
lali=[]
for i in range(16):
    lali.append(testDataAuth.bootstrap_config[i])
print(lali)

[0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 1, 0, 1, 0]


In [71]:
testDataAuth.bootstrap_config_len

16

In [134]:
testDataAuth=ffi.new('AuthGranted *',grantedAuth[0])

In [93]:
ffi.string(myAuth.app.id)

b'ridsTest'

In [63]:
grantedAuth.access_container_info.id

<cdata 'unsigned char[32]' 0x7ffd1d44c6e0>

In [64]:
type(grantedAuth.access_container_info.tag)

int

In [65]:
grantedAuth.access_container_info

<cdata 'AccessContInfo &' 0x7ffd1d44c6e0>

In [66]:
grantedAuth.access_container_info.tag

15000

In [67]:
grantedAuth.access_container_entry.containers_len

0

In [68]:
grantedAuth.access_container_info.nonce

<cdata 'unsigned char[24]' 0x7ffd1d44c708>

In [69]:
grantedAuth.bootstrap_config_len

16

In [70]:
grantedAuth.bootstrap_config_cap

16

---

In [65]:
grantedAuth

<cdata 'AuthGranted *' owning 336 bytes>

In [66]:
grantedAuth.access_container_info.id

<cdata 'unsigned char[32]' 0x560214c901e0>

In [67]:
type(grantedAuth.access_container_info.tag)

int

In [68]:
grantedAuth.access_container_info

<cdata 'AccessContInfo &' 0x560214c901e0>

In [69]:
grantedAuth.access_container_info.tag

15000

In [70]:
grantedAuth.access_container_entry.containers_len

0

In [71]:
grantedAuth.access_container_info.nonce

<cdata 'unsigned char[24]' 0x560214c90208>

In [72]:
grantedAuth.bootstrap_config_len

16

In [73]:
grantedAuth.bootstrap_config_cap

16

## looking into the granted authentication

In [58]:
def printArray(array):
    arr=[]
    for element in array:
        arr.append(element)
    print(arr)

In [59]:
grantedAuth.app_keys.owner_key

<cdata 'unsigned char[32]' 0x7ffea2566a90>

In [56]:
len(grantedAuth.app_keys.owner_key)

32

In [59]:
printArray(grantedAuth.app_keys.owner_key)

[176, 107, 43, 207, 210, 127, 0, 0, 8, 235, 13, 192, 210, 127, 0, 0, 176, 107, 43, 207, 210, 127, 0, 0, 48, 238, 38, 207, 210, 127, 0, 0]


In [62]:
len(grantedAuth.app_keys.enc_key)

32

In [61]:
len(grantedAuth.app_keys.sign_pk)

32

In [62]:
len(grantedAuth.app_keys.sign_sk)

64

In [63]:
len(grantedAuth.app_keys.enc_pk)

32

In [64]:
len(grantedAuth.app_keys.enc_sk)

32